## 载入序列数据

In [27]:
import pandas as pd

In [28]:
events_df = pd.read_csv('./data/retailrocket/events.csv')
category_tree_df = pd.read_csv('./data/retailrocket/category_tree.csv')
item_properties_1_df = pd.read_csv('./data/retailrocket/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('./data/retailrocket/item_properties_part2.csv')

In [29]:
item_properties_df = pd.concat([item_properties_1_df, item_properties_2_df])

In [ ]:
events_df.head()

删除event和transactionid无用信息

In [31]:
events_df = events_df.drop(['event', 'transactionid'],axis=1)

按照时间和id排序

In [32]:
events_df = events_df.sort_values(['visitorid', 'timestamp'])

In [ ]:
events_df

In [ ]:
events_df = events_df.reset_index(drop=True)
events_df

drop掉行动序列小于3的

In [21]:
users = events_df['visitorid']
users = users.value_counts()
users = users.iloc[users.values > 2].index
users = users.sort_values()
users

Int64Index([      0,       2,       6,       7,      23,      37,      51,
                 54,      60,      64,
            ...
            1407503, 1407504, 1407511, 1407512, 1407513, 1407526, 1407527,
            1407528, 1407530, 1407573],
           dtype='int64', length=200028)

In [35]:
group = events_df.groupby('visitorid',group_keys=False).groups

In [40]:
len(group)

1407580

In [69]:
user_list=[]
action_list=[]
all_items=[]
for user in events_df.visitorid.unique().tolist():
    item_list=[]
    for item in group[user]:  # 将user访问过的不重复的item加入到item列表中
        item = events_df['itemid'][item]
        #print(item not in item_list)
        if item not in item_list:
            item_list.append(item)
    if len(item_list) > 3: # 如果访问过的不重复的物品序列长度大于k则加入到最终的数据集中
        user_list.append(user)
        action_list.append(item_list)
        all_items += item_list

In [70]:
len(user_list)

63294

In [71]:
len(all_items)

521872

In [74]:
item_set = set(all_items)
len(item_set)

103218

In [84]:
item2id = dict(zip(item_set, range(len(item_set))))

把actionlist里的itemid改为新规范过的id

In [80]:
id_action_list=[]
for lst in action_list:
    ids=[]
    for item in lst:
        item = item2id[item]
        ids.append(item)
    id_action_list.append(ids)

In [82]:
data = {'user':user_list, 'actions':id_action_list}
df = pd.DataFrame(data)

In [83]:
df

,user,actions
0,2,"[27951, 102740, 93055, 35878]"
1,51,"[42809, 87860, 73955, 22139, 7477]"
2,54,"[55906, 9268, 17288, 25489, 79585, 100361]"
3,64,"[62800, 14865, 96783, 52590, 94869, 71226]"
4,74,"[20064, 26393, 25324, 103214, 24745]"
...,...,...
63289,1407437,"[56666, 49145, 15467, 51064]"
63290,1407476,"[43100, 77981, 33107, 91130, 102364]"
63291,1407479,"[60040, 73148, 42512, 72572, 23216, 48534]"
63292,1407512,"[47890, 45389, 28656, 23983, 32329, 12684, 603..."


两个任务：
1. 将在序列中的itemid映射到[1, item_size]的区间中
2. 查看大小